In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


### Importing the data

In [2]:
users_history = pd.read_csv("/kaggle/input/million-song-dataset-spotify-lastfm/User Listening History.csv")
music_info  = pd.read_csv("/kaggle/input/million-song-dataset-spotify-lastfm/Music Info.csv")

### Grouping the songs of each user

In [3]:
user_song_list = users_history.groupby('user_id', observed=True)[['track_id', 'playcount']].apply(lambda x: list(zip(x['track_id'], x['playcount']))).to_dict()

###  Removing the users who have listened to less than 50 songs

In [4]:
user_song_list = {user: songs for user, songs in user_song_list.items() if len(songs) >= 50}

### Removing the data related to users that have listened to less than 50 songs

In [5]:
users_history = users_history[users_history['user_id'].isin(user_song_list.keys())] 

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

### Concatenating all the features what have string values into a single one

In [7]:
music_info[['tags', 'artist', 'genre', 'name']] = music_info[['tags', 'artist', 'genre', 'name']].fillna('')
music_info['combined_features'] = music_info[['tags', 'artist', 'genre', 'name']].apply(lambda x: ' '.join(x), axis=1)

### Creating Tf/Idf matrix

In [8]:
# Apply TF-IDF to the combined features
tfidf = TfidfVectorizer(max_features=2000)
tfidf_matrix = tfidf.fit_transform(music_info['combined_features'])


In [9]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=music_info['track_id'], columns=tfidf.get_feature_names_out())

In [10]:
tfidf_df.shape

(50683, 2000)

### Splitting the data

In [11]:
# Split the users_history data into train and test sets
train, test = train_test_split(users_history, test_size=0.2, random_state=42)

# Create dictionaries for train and test data
user_train_data = train.groupby('user_id', observed=True)[['track_id', 'playcount']].apply(lambda x: list(zip(x['track_id'], x['playcount']))).to_dict()
user_test_data = test.groupby('user_id', observed=True)[['track_id', 'playcount']].apply(lambda x: list(zip(x['track_id'], x['playcount']))).to_dict()

In [12]:
def build_user_profiles(user_train_data, tfidf_df):
    user_profiles = {}
    for user_id, tracks in user_train_data.items():
        user_vector = np.zeros(tfidf_df.shape[1])
        for track_id, playcount in tracks:
            if track_id in tfidf_df.index:
                user_vector += tfidf_df.loc[track_id].values * playcount
        user_profiles[user_id] = user_vector / np.linalg.norm(user_vector)  # Normalize the user profile vector
    return user_profiles


### Building user profiles

In [13]:
user_profiles = build_user_profiles(user_train_data, tfidf_df)

In [14]:
def recommend_songs_content_based(user_id, user_profiles, tfidf_df, user_train_data, n_recommendations=5):
    if user_id not in user_profiles:
        print(f"User ID {user_id} not found in the training data.")
        return []

    user_profile = user_profiles[user_id]
    cosine_similarities = cosine_similarity([user_profile], tfidf_df.values).flatten()
    similar_indices = cosine_similarities.argsort()[::-1]
    recommended_tracks = [tfidf_df.index[i] for i in similar_indices if tfidf_df.index[i] not in [track_id for track_id, _ in user_train_data[user_id]]]

    return recommended_tracks[:n_recommendations]

In [15]:
def evaluate_single_user_content_based(user_id, user_train_data, user_test_data, user_profiles, tfidf_df, n_recommendations):
    if user_id not in user_test_data:
        print(f"User ID {user_id} not found in the test data.")
        return None, None

    true_tracks = [track for track, _ in user_test_data[user_id]]
    recommended_tracks = recommend_songs_content_based(user_id, user_profiles, tfidf_df, user_train_data, n_recommendations)

    true_positives = len(set(recommended_tracks) & set(true_tracks))
    precision = true_positives / len(recommended_tracks) if recommended_tracks else 0
    recall = true_positives / len(true_tracks) if true_tracks else 0

    return precision, recall


In [16]:
def evaluate_model_content_based(user_test_data, user_train_data, user_profiles, tfidf_df, n_recommendations):
    total_precision = 0
    total_recall = 0
    user_count = 0

    for user_id in user_test_data:
        precision, recall = evaluate_single_user_content_based(user_id, user_train_data, user_test_data, user_profiles, tfidf_df, n_recommendations)
        if precision is not None and recall is not None:
            total_precision += precision
            total_recall += recall
            user_count += 1
            if user_count%50 == 0:
                print(user_count)

    avg_precision = total_precision / user_count if user_count else 0
    avg_recall = total_recall / user_count if user_count else 0

    return avg_precision, avg_recall


In [17]:
dict(list(user_song_list.items())[:1])

{'0003a64f7a69e5b87a80b09c3772227185c235c7': [('TRRCWXH128F42B917C', 1),
  ('TRMHZLT12903CEA646', 1),
  ('TROVIQH128F42B91A1', 1),
  ('TRDNHAW128F429DB9A', 1),
  ('TRBVSQT128F427A82F', 1),
  ('TRAFUNV128F92CFEB2', 2),
  ('TRRXFHO128EF3550BC', 1),
  ('TRKQLRS128F4230F04', 1),
  ('TRBVNWT128F93173BA', 1),
  ('TRYYDLF128F423BB17', 1),
  ('TRUWRWK128F92E16B6', 1),
  ('TRMPTXE128F9303E92', 1),
  ('TRNGKOQ128E0780DAC', 1),
  ('TRVKPXM128F4284E60', 1),
  ('TRHTYWL128F429EFD8', 1),
  ('TRBROBA128F9321BC2', 3),
  ('TRFNDHN128F426BE40', 1),
  ('TRAVRKY128F429EA68', 1),
  ('TRILTQH128F93264C4', 1),
  ('TRUTKSF128F147B0E2', 1),
  ('TRBXLFU128F427A494', 1),
  ('TRRUZLF128F42396D0', 1),
  ('TRBMCGS128F92D2C27', 5),
  ('TROBYUK128F9335CA7', 1),
  ('TRKRADU128F92E17F6', 1),
  ('TRJGWMZ128F149F0BD', 1),
  ('TRGVKBK128F429EA2D', 1),
  ('TRIFCFC128F4230F13', 1),
  ('TRJLVOB128F931EB19', 1),
  ('TRJDWKM128E07817F1', 1),
  ('TRNPEXS128F1460294', 5),
  ('TRCYSEC128F428B5E8', 1),
  ('TRTBNOP12903CDB267', 6),

### Trying the function on a single user

In [18]:
avg_precision, avg_recall = evaluate_single_user_content_based('0003a64f7a69e5b87a80b09c3772227185c235c7', user_test_data, user_train_data, user_profiles, tfidf_df, 44)
print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")


Average Precision: 0.06818181818181818
Average Recall: 0.06382978723404255


### Evaluating the model for all users

In [19]:
avg_precision, avg_recall = evaluate_model_content_based(user_test_data, user_train_data, user_profiles, tfidf_df, 44)
print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150
1